<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
!pip3 install folium
!pip3 install wget

In [1]:
import wget
from folium import plugins
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
import folium

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [24]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.


In [25]:
spacex_df.head(2)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356


In [26]:
spacex_df.groupby(['Launch Site'], as_index=False).first()

,Launch Site,Flight Number,Date,Time (UTC),Booster Version,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,CCAFS LC-40,1,2010-06-04,18:45:00,F9 v1.0 B0003,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,CCAFS SLC-40,45,2017-12-15,15:36:00,F9 FT B1035.2,SpaceX CRS-13,2205.0,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
2,KSC LC-39A,30,2017-02-19,14:39:00,F9 FT B1031.1,SpaceX CRS-10,2490.0,LEO (ISS),NASA (CRS),Success (ground pad),1,28.573255,-80.646895
3,VAFB SLC-4E,6,2013-09-29,16:00:00,F9 v1.1 B1003,CASSIOPE,500.0,Polar LEO,MDA,Uncontrolled (ocean),0,34.632834,-120.610745


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [80]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [81]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [70]:
for lat, lng,base, in zip(launch_sites_df['Lat'], launch_sites_df['Long'],launch_sites_df['Launch Site']):
    print(lat, lng,base)

28.56230197 -80.57735648 CCAFS LC-40
28.56319718 -80.57682003 CCAFS SLC-40
28.57325457 -80.64689529 KSC LC-39A
34.63283416 -120.6107455 VAFB SLC-4E


In [82]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()


# loop through the 100 crimes and add each to the incidents feature group
for lat, lng,base, in zip(launch_sites_df['Lat'], launch_sites_df['Long'],launch_sites_df['Launch Site']):
    label_to_pop_up="Base name:: "+base
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, color='#000000', fill=True,
            popup=label_to_pop_up,
            tooltip=label_to_pop_up#esto hace de mousehover
        )
    ).add_to(site_map)
    

# add pop-up text to each marker on the map
latitudes = list(launch_sites_df['Lat'])
longitudes = list(launch_sites_df['Long'])
labels = list(launch_sites_df['Launch Site'])

for lat, lng,base, in zip(launch_sites_df['Lat'], launch_sites_df['Long'],launch_sites_df['Launch Site']):
    folium.map.Marker([lat, lng],popup=base, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % base, )).add_to(site_map)    
    

site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [27]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


In [28]:
Mapping = {1 : "green",0 : "red"}

spacex_df["marker_colorJavi2"] = spacex_df["class"].map(Mapping)#mapear
spacex_df.tail()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_colorJavi2
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.0,GTO,SES,Controlled (ocean),0,28.563197,-80.57682,red
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.0,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.57682,red
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.0,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.57682,red
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.0,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.57682,green
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.0,GTO,SES,No attempt,0,28.563197,-80.57682,red


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [29]:
marker_cluster = MarkerCluster()


In [42]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [31]:
marker_color=[]
for i,pos in enumerate(spacex_df.values.tolist()):
    base,lat,long,clase=[pos[4],pos[11],pos[12],pos[10]]
    if clase==1:
        marker_color.append("green")
    elif clase==0:
        marker_color.append("red")
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_colorJavi']=marker_color

In [35]:
False not in (spacex_df['marker_colorJavi2']==spacex_df['marker_color']).values.tolist()

True

In [32]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_colorJavi2,marker_colorJavi,marker_color
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895,green,green,green
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895,green,green,green
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895,green,green,green
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820,green,green,green
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820,green,green,green
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820,red,red,red
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820,red,red,red
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820,red,red,red
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820,green,green,green
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820,red,red,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [21]:
for index, record in spacex_df.iterrows():
    print(index,record.values[0])

0 CCAFS LC-40
1 CCAFS LC-40
2 CCAFS LC-40
3 CCAFS LC-40
4 CCAFS LC-40
5 CCAFS LC-40
6 CCAFS LC-40
7 CCAFS LC-40
8 CCAFS LC-40
9 CCAFS LC-40
10 CCAFS LC-40
11 CCAFS LC-40
12 CCAFS LC-40
13 CCAFS LC-40
14 CCAFS LC-40
15 CCAFS LC-40
16 CCAFS LC-40
17 CCAFS LC-40
18 CCAFS LC-40
19 CCAFS LC-40
20 CCAFS LC-40
21 CCAFS LC-40
22 CCAFS LC-40
23 CCAFS LC-40
24 CCAFS LC-40
25 CCAFS LC-40
26 VAFB SLC-4E
27 VAFB SLC-4E
28 VAFB SLC-4E
29 VAFB SLC-4E
30 VAFB SLC-4E
31 VAFB SLC-4E
32 VAFB SLC-4E
33 VAFB SLC-4E
34 VAFB SLC-4E
35 VAFB SLC-4E
36 KSC LC-39A
37 KSC LC-39A
38 KSC LC-39A
39 KSC LC-39A
40 KSC LC-39A
41 KSC LC-39A
42 KSC LC-39A
43 KSC LC-39A
44 KSC LC-39A
45 KSC LC-39A
46 KSC LC-39A
47 KSC LC-39A
48 KSC LC-39A
49 CCAFS SLC-40
50 CCAFS SLC-40
51 CCAFS SLC-40
52 CCAFS SLC-40
53 CCAFS SLC-40
54 CCAFS SLC-40
55 CCAFS SLC-40


In [22]:
label_to_pop_up

'Base name:: VAFB SLC-4E'

In [37]:
spacex_df.head(1)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_colorJavi2,marker_colorJavi,marker_color
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356,red,red,red


In [83]:
# Add marker_cluster to current site_map
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)



# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for i,pos in enumerate(spacex_df.values.tolist()):
    base,lat,long,clase,rowmarker,Date,Time,payload,landing_out=[pos[4],pos[11],pos[12],pos[10],pos[13],pos[1],pos[2],pos[5],pos[9]]
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    label_to_pop_up="Base name: "+base+" Date: "+Date+"//"+Time+" Payload: "+payload+" Landing Outcome: "+landing_out
    marker=folium.Marker(
        location=[lat, long],
        icon=folium.Icon(color='white', icon_color=rowmarker),
        popup=base,
        tooltip=label_to_pop_up#esto hace de mousehover
    )
    marker_cluster.add_child(marker)


site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [73]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [43]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [54]:
set(spacex_df["Lat"].values.tolist())

{28.56230197, 28.56319718, 28.57325457, 34.63283416}

In [55]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_lat=28.57325457
launch_site_lon=-80.64689529
coastline_lat=28.57666
coastline_lon=-80.65553
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [76]:
distance_coastline=round(distance_coastline,2)
distance_coastline

0.92

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [78]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
    [coastline_lat,coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [84]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates=[[launch_site_lat,launch_site_lon],[coastline_lat,coastline_lon]]
lines=folium.PolyLine(locations=coordinates, weight=1)
distance_marker.add_to(site_map)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [85]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_lat=28.57325457
launch_site_lon=-80.64689529
coastline_lat=28.40046
coastline_lon=-80.60909
distance_coastline = round(calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon),2)
distance_coastline

19.57

In [86]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
    [coastline_lat,coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates=[[launch_site_lat,launch_site_lon],[coastline_lat,coastline_lon]]
lines=folium.PolyLine(locations=coordinates, weight=1)
distance_marker.add_to(site_map)
site_map.add_child(lines)

In [92]:
spacex_csv_file = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df=pd.read_csv(spacex_csv_file)
del spacex_df["Unnamed: 0"]
spacex_df

100% [................................................................................] 2476 / 2476

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.00,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.00,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.00,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.00,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.00,F9 v1.0 B0007,v1.0
5,7,CCAFS LC-40,0,3170.00,F9 v1.1,v1.1
6,8,CCAFS LC-40,0,3325.00,F9 v1.1,v1.1
7,9,CCAFS LC-40,0,2296.00,F9 v1.1,v1.1
8,10,CCAFS LC-40,0,1316.00,F9 v1.1,v1.1
9,11,CCAFS LC-40,0,4535.00,F9 v1.1,v1.1


In [99]:
sites=spacex_df["Launch Site"].unique().tolist()
sites.insert(0, "All Sites")
sites

['All Sites', 'CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [100]:
sites=spacex_df["Launch Site"].unique().tolist()
sites.insert(0, "All Sites")
labels=[]
for i,pos in enumerate(sites):
    if pos=="All Sites":
        labels.append({'label': pos, 'value': "ALL"})
    else:
        labels.append({'label': pos, 'value': pos})
labels

[{'label': 'All Sites', 'value': 'ALL'},
 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}]

In [93]:
  dcc.Dropdown(id='id',
                options=[
                    {'label': 'All Sites', 'value': 'ALL'},
                    {'label': 'site1', 'value': 'site1'},
                ],
                value='ALL',
                placeholder="place holder here",
                searchable=True
                ),

100% [................................................................................] 2110 / 2110

In [2]:
import pandas as pd
import wget
from folium import plugins
import numpy as np  # useful for many scientific computing in Python
import folium
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import numpy as np
from scipy import stats
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)
Mapping = {1 : "green",0 : "red"}

spacex_df["marker_colorJavi2"] = spacex_df["class"].map(Mapping)#mapear
spacex_df.tail()

100% [................................................................................] 7710 / 7710

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_colorJavi2
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.0,GTO,SES,Controlled (ocean),0,28.563197,-80.57682,red
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.0,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.57682,red
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.0,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.57682,red
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.0,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.57682,green
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.0,GTO,SES,No attempt,0,28.563197,-80.57682,red


In [59]:
l=[3000,9000]
df_can=spacex_df
df_can[(df_can['Payload Mass (kg)']>=l[0]) & (df_can['Payload Mass (kg)']<=l[1])]

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_colorJavi2
5,7,2013-12-03,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170.00,GTO,SES,No attempt,0,28.562302,-80.577356,red
6,8,2014-01-06,22:06:00,F9 v1.1,CCAFS LC-40,Thaicom 6,3325.00,GTO,Thaicom,No attempt,0,28.562302,-80.577356,red
9,11,2014-08-05,8:00:00,F9 v1.1,CCAFS LC-40,AsiaSat 8,4535.00,GTO,AsiaSat,No attempt,0,28.562302,-80.577356,red
10,12,2014-09-07,5:00:00,F9 v1.1 B1011,CCAFS LC-40,AsiaSat 6,4428.00,GTO,AsiaSat,No attempt,0,28.562302,-80.577356,red
14,16,2015-03-02,3:50:00,F9 v1.1 B1014,CCAFS LC-40,ABS-3A Eutelsat 115 West B,4159.00,GTO,ABS Eutelsat,No attempt,0,28.562302,-80.577356,red
16,18,2015-04-27,23:03:00,F9 v1.1 B1016,CCAFS LC-40,TürkmenÄlem 52°E / MonacoSAT,4707.00,GTO,Turkmenistan National Space Agency,No attempt,0,28.562302,-80.577356,red
19,22,2016-03-04,23:35:00,F9 FT B1020,CCAFS LC-40,SES-9,5271.00,GTO,SES,Failure (drone ship),0,28.562302,-80.577356,red
20,23,2016-04-08,20:43:00,F9 FT B1021.1,CCAFS LC-40,SpaceX CRS-8,3136.00,LEO (ISS),NASA (CRS),Success (drone ship),1,28.562302,-80.577356,green
21,24,2016-05-06,5:21:00,F9 FT B1022,CCAFS LC-40,JCSAT-14,4696.00,GTO,SKY Perfect JSAT Group,Success (drone ship),1,28.562302,-80.577356,green
22,25,2016-05-27,21:39:00,F9 FT B1023.1,CCAFS LC-40,Thaicom 8,3100.00,GTO,Thaicom,Success (drone ship),1,28.562302,-80.577356,green


In [58]:
bins=[]
for i in range(0,4):
    if i==3:
        bins.append(np.array_split(range(0,10000), 4)[i][0])
        bins.append(np.array_split(range(0,10000+1), 4)[3][-1])
        
    else:
        bins.append(np.array_split(range(0,10000), 4)[i][0])
my_dict = {bins[0]:str(bins[0])}
for i in bins:
    my_dict[i]=str(i)
my_dict  

{0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'}

In [41]:
print(np.array_split(range(0,10000+1), 4)[3][-1])

10000


In [18]:
dx=spacex_df.groupby('Launch Site')['class'].sum()
dx1=dx.nlargest(len(dx)).reset_index(name='Count')
fig = px.pie(dx1, values="Count", names='Launch Site', title='Total succes rate by sites')

,Launch Site,Count
0,KSC LC-39A,10
1,CCAFS LC-40,7
2,VAFB SLC-4E,4
3,CCAFS SLC-40,3


In [19]:
fig = px.pie(dx1, values="Count", names='Launch Site', title='Total succes rate by sites')

In [26]:
filtered_df = spacex_df[spacex_df['Launch Site']=="CCAFS LC-40"]
dx = filtered_df.groupby(["class"]).size().reset_index(name='Count')
fig = px.pie(dx, values="Count", names='class', title='Total succes rate by sites')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.pie(dx1, values="Count", names='Launch Site', title='Total succes rate by sites')
fig

In [5]:
dx=spacex_df[spacex_df["class"]==1]
dx1 = dx.groupby(["Launch Site"]).size().reset_index(name='Count')#conteo groupby de dataframe, ValueCounts, valores unicos
dx1

,Launch Site,Count
0,CCAFS LC-40,7
1,CCAFS SLC-40,3
2,KSC LC-39A,10
3,VAFB SLC-4E,4


In [ ]:

app = dash.Dash(__name__)

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# Read the automobiles data into pandas dataframe

df_total=spacex_df


ops=df_total['Launch Site'].unique().tolist()
ops.append("All Sites")
                                                                                            
app = dash.Dash()
                            
app.layout = html.H2('Space X Launch Record Dashboard', style={'textAlign':'center', 'color': '#008838'}),


                                html.Div([html.H2('Launch Site:', style={'color': '#02B3D5', 'margin-right': '2em'}), dcc.Dropdown(options=ops,value='All Sites', id='input-wheels',style={'textAlign':'center','height':'40px', 'font-size': 20,'width':350}),], 
                                style={'margin-right': '2em','font-size': 20}),
                                 
                                html.H2('Total Succes rate by sites', 
                                style={'textAlign':'center', 'color': '#008838'}),
                                html.Div([
                                        html.Div(dcc.Graph(id='bar1-plot')),
                                        
                                ]),
                                
                                html.Br(),
                                html.Div([html.H2('Launch Site:', style={'color': '#02B3D5', 'margin-right': '2em'}), dcc.Dropdown(options=ops,value='All Sites', id='input-wheels',style={'textAlign':'center','height':'40px', 'font-size': 20,'width':350}),], 
                                style={'margin-right': '2em','font-size': 20}),
                                html.Div([
                                    html.H3('Current Month Income vs Average Month Income', 
                                style={'textAlign':'center', 'color': '#008838'}),
                                        html.Div(dcc.Graph(id='bar7-plot')),
                                        html.Div(dcc.Graph(id='bar8-plot')),
                                ], style={'display': 'flex'}), 
                                html.Br(),
                                html.H2('Brand Price Analyzer by Features, pick 1 or 2 features to be compared vs average price', style={'textAlign':'center', 'color': '#008838'}),
                                html.Div([html.H2('Comparing features', style={'color': '#02B3D5','margin-right': '2em'}), dcc.Dropdown(options=['car_seniority','transmission','Wheel drive','Type of Fuel','seatingCapacity','bodyType','Number of doors','NO_Feature1'],value='bodyType', id='input-wheels2',style={'textAlign':'center','height':'40px', 'font-size': 20,'width':350}),], 
                                style={'margin-right': '2em','font-size': 20}),
                                html.Div([html.H2('Comparing features', style={'color': '#02B3D5','margin-right': '2em'}), dcc.Dropdown(options=['car_seniority','transmission','Wheel drive','Type of Fuel','seatingCapacity','bodyType','Number of doors','NO_Feature2'],value='NO_Feature2', id='input-wheels3',style={'textAlign':'center','height':'40px', 'font-size': 20,'width':350}),], 
                                style={'margin-right': '2em','font-size': 20}),
                                html.Br(),
                                html.Div([
                                        html.Div(dcc.Graph(id='bar6-plot')),
                                        
                                ]),
                                ])

# add callback decorator
@app.callback( [Output(component_id='pie1-plot', component_property='figure'),Output(component_id='bar2-plot', component_property='figure'),Output(component_id='bar3-plot', component_property='figure'),Output(component_id='bar4-plot', component_property='figure'),Output(component_id='bar5-plot', component_property='figure'),Output(component_id='bar6-plot', component_property='figure'),Output(component_id='bar7-plot', component_property='figure'),Output(component_id='bar8-plot', component_property='figure')],#viene conjuntamente con la funcion de abajo, la cual tiene como parametro
               [Input(component_id='input-wheels', component_property='value'),Input(component_id='input-wheels2', component_property='value'),Input(component_id='input-wheels3', component_property='value')])#como parametro input el input componente id indicado, y de output, lo que se retorna en la funcion

# Add computation to callback function and return graph
def get_graph(entered_wheel,entered_wheel2,entered_wheel3):
    # Select 2019 data
    ddf=df_total[df_total['make']==entered_wheel]
    ddf = ddf[ddf['transmission'].notnull()]#this is to avoid error with graph number 4
    #ddf=ddf[ddf['model']!=""]#quito modelos sin nombre

    minn=df_total[['price']].sort_values(by=["price"],ascending=False).values.tolist()[-1]
    maxx=df_total[['price']].sort_values(by=["price"],ascending=False).values.tolist()[3]
    fig = px.box(ddf, y="price")
    fig.update_traces(fillcolor='rgb(55, 83, 109)',marker=dict(size=2,color='rgb(7, 183, 216)'))
    
    fig2 = px.histogram(ddf, x="price",nbins=18)
    fig2.update_xaxes(range = [minn[0],maxx[0]])
    for i in range(len(fig2.data)):
        fig2.data[i].marker.line.width = 3
        fig2.data[i].marker.line.color = "rgb(7, 183, 216)"
        fig2.data[i].marker.color = "rgb(55, 83, 109)"

    labels=[]
    for i,pos in enumerate(sorted(ddf['model'].unique())):
                labels.append(round(ddf[ddf["model"]==pos]["price"].mean(),2))
                labels.append(pos)
    dx= pd.DataFrame(np.asarray(labels).reshape(-1,2),columns=['price','model'])
    dx['price']=dx['price'].astype('float')
    fig3 = px.bar(dx,x="model", y="price",barmode = 'group')#,width=1450, height=605
        #fig.update_xaxes(tickvals=np.arange(7), ticktext=dx['price'].values.tolist())
    for i, t in enumerate([dx['price'].values.tolist()]):
                fig3.data[i].text = t
                fig3.data[i].textposition = 'outside'
    for i in range(len(fig3.data)):
        fig3.data[i].marker.line.width = 3
        fig3.data[i].marker.line.color = "rgb(0,80,181)"
        fig3.data[i].marker.color = "rgb(255,252,201)"

    fig4 = px.scatter(ddf[ddf['transmission'] != ''], x="year", y="price", color='mileage',facet_col="transmission",width=1450, height=555)#,width=1450, height=605

#groupby que explican estadisticos de un groupby, se explica los estadisticos de college para la columna wasitgoal
    candlestick_data = ddf.groupby(["model"], as_index=False).agg({"price": [lambda x:stats.mode(x)[0],'mean','min', 'max', 'first', 'last','sum']}).round(2)


    candlestick_data.columns.set_levels(['mode','mean','min', 'max', 'first', 'last','sum',""],level=1,inplace=True)
    candlestick_data.columns.set_levels(['price','car','Drive Type',],level=0,inplace=True)
    fig5 = go.Figure(data=[go.Candlestick(x=candlestick_data['car'],
                open=candlestick_data['price']['first'], 
                high=candlestick_data['price']['max'],
                low=candlestick_data['price']['min'], 
                close=candlestick_data['price']['last'])
                ])
    cs = fig5.data[0]

# Set line and fill colors
    cs.increasing.fillcolor = 'rgb(255,252,201)'

    cs.increasing.line.color = 'rgb(0,80,181)'
                      

    fig5.update_layout(xaxis_rangeslider_visible=False)

    
    ddf['seatingCapacity']=ddf['seatingCapacity'].astype(str)
    ddf['car_seniority']=ddf['car_seniority'].astype(str)
    if entered_wheel2=="":
        entered_wheel2="NO_Feature1"
    if entered_wheel3=="":
        entered_wheel3="NO_Feature2"
    
    
    if (entered_wheel3=="NO_Feature2" and entered_wheel2=="NO_Feature1") or (entered_wheel2==entered_wheel3):
        data2=ddf.groupby(['bodyType'])['price'].mean().round(2).reset_index()#bodyType
        data2=data2[data2['bodyType'] != '']
        fig6 = px.bar(data2, x="bodyType", y="price",
                        barmode = 'group',title="Pick at least one Feature, to get results like this one: ",width=1450, height=555)#,width=1450, height=555

        fig6.data[0].text = data2["price"].values.tolist()
        fig6.data[0].textposition = 'outside'

        for i in range(len(fig6.data)):
                fig6.data[i].marker.line.width = 4
                fig6.data[i].marker.line.color = "#008838"
                fig6.data[i].marker.color = "#02B3D5"    
    elif entered_wheel2=="NO_Feature1":
        data2=ddf.groupby([entered_wheel3])['price'].mean().round(2).reset_index()#bodyType
        data2=data2[data2[entered_wheel3] != '']
        fig6 = px.bar(data2, x=entered_wheel3, y="price",
                        barmode = 'group',width=1450, height=555)#,width=1450, height=555

        fig6.data[0].text = data2["price"].values.tolist()
        fig6.data[0].textposition = 'outside'
        for i in range(len(fig6.data)):
                fig6.data[i].marker.line.width = 4
                fig6.data[i].marker.line.color = "#008838"
                fig6.data[i].marker.color = "#02B3D5"  
        fig6.update_layout(title={'text': '<b>Results for '+str(entered_wheel)+'</b>'})
    elif entered_wheel3=="NO_Feature2":
        data2=ddf.groupby([entered_wheel2])['price'].mean().round(2).reset_index()#bodyType
        data2=data2[data2[entered_wheel2] != '']
        fig6 = px.bar(data2, x=entered_wheel2, y="price",
                        barmode = 'group',width=1450, height=555)#,width=1450, height=555

        fig6.data[0].text = data2["price"].values.tolist()
        fig6.data[0].textposition = 'outside'
        for i in range(len(fig6.data)):
                fig6.data[i].marker.line.width = 4
                fig6.data[i].marker.line.color = "#008838"
                fig6.data[i].marker.color = "#02B3D5"  
        fig6.update_layout(title={'text': '<b>Results for '+str(entered_wheel)+'</b>'})
    else:
        data2=ddf.groupby([entered_wheel2,entered_wheel3])['price'].mean().round(2).reset_index()#bodyType
        data2=data2[(data2[entered_wheel2]!='') & (data2[entered_wheel3] != '')]
        fig6 = px.bar(data2, x=entered_wheel2, y="price",
                        color=entered_wheel3, barmode = 'group',width=1450, height=555)#,width=1450, height=555

        labels=[]
        for i,pos in enumerate(data2[entered_wheel3].unique()):
                labels.append(data2[data2[entered_wheel3]==pos]["price"].values.tolist())


        for i, t in enumerate(labels):
                fig6.data[i].text = t
                fig6.data[i].textposition = 'outside'
        fig6.update_layout(uniformtext_minsize=11.5, uniformtext_mode='hide')
        colors=['#02B3D5','#008838','#101820', '#0085CA','#BFC0BF', '#001489','#A71930','#FFB612','#4f2683','#125740','#69BE28','#203731','#B3995D','#FFB612','#FB4F14','pink','brown','black','yellow','purple','green','cyan','blue','red']

        for i2,pos in enumerate(colors):
                if i2<len(data2[entered_wheel3].unique().tolist()):
                    fig6.data[i2].marker.line.width = 3
                    
                    fig6.data[i2].marker.line.color = "white"
                    fig6.data[i2].marker.color = pos
        fig6.update_layout(title={'text': '<b>Results for '+str(entered_wheel)+'</b>'})
        

    data1 = ddf.set_index('vehicleAddedDate') 
    Precio_Coche_mes=data1.resample('m').sum().loc[:,['price']].round(2)
    data1 = df_total.set_index('vehicleAddedDate') 
    Precio_Coche_mes=Precio_Coche_mes.reset_index()

    Precio_Coche_mes['year'] = pd.DatetimeIndex(Precio_Coche_mes['vehicleAddedDate']).year
    Precio_Coche_mes['month'] = pd.DatetimeIndex(Precio_Coche_mes['vehicleAddedDate']).month
    Mapping = {1 : "January",2 : "February",3 : "March",4 : "April",
            5 : "May",6 : "June",7 : "July",8 : "August",9 : "September",10 : "October",11 : "November",12 : "December"}

    Precio_Coche_mes["Month"] = Precio_Coche_mes["month"].map(Mapping)
    Precio_Coche_mes["Sales_month"]=Precio_Coche_mes["Month"]+"_"+Precio_Coche_mes["year"].astype(str)
    maxincome=max(Precio_Coche_mes['price'])
    valuemean=round(Precio_Coche_mes['price'].mean(),2)
    value=Precio_Coche_mes['price'].values.tolist()[-1]
    if value>=valuemean:
        fig7 = go.Figure(go.Indicator(
            mode = "number+gauge+delta", value = value,
            delta = {'reference': valuemean, 'position': "top"},
            gauge = {
                'shape': "angular",
                'axis': {'range': [0, maxincome]},
                'threshold': {
                    'line': {'color': "yellow", 'width': 3},
                    'thickness': 0.75, 'value': valuemean},
                'bgcolor': "white",
                'steps': [
                    {'range': [0, maxincome], 'color': "#9DFE9D"}],
                'bar': {'color': "#00A350"}}))
    else:
        fig7 = go.Figure(go.Indicator(
            mode = "number+gauge+delta", value = value,
            delta = {'reference': valuemean, 'position': "top"},
            gauge = {
                'shape': "angular",
                'axis': {'range': [0, maxincome]},
                'threshold': {
                    'line': {'color': "yellow", 'width': 3},
                    'thickness': 0.75, 'value': valuemean},
                'bgcolor': "white",
                'steps': [
                    {'range': [0, maxincome], 'color': "#ffcccb"}],
                'bar': {'color': "#F7330E"}}))
    fig7.add_annotation(x=valuemean/(maxincome), y=1.05,
                text="Mean Montly Average Overall",
            showarrow=True,
            font=dict(
                family="Courier New, monospace",
                size=12,
                color="yellow"
                ),
            align="center",
            ax=0,
            ay=0,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="#ff7f0e",
            opacity=0.8
            )
    fig7.add_annotation(x=0.9, y=0,
                text="Highest Month Income",
            showarrow=True,
            font=dict(
                family="Calibri",
                size=11.15,
                color="black"
                ),
            align="center",
            arrowhead=2,
            #arrowsize=3,
            arrowwidth=1.20,
            ax=-85,
            ay=-0,
            opacity=0.65
            )

    datas=ddf.groupby(['available']).size().reset_index(name='Count')#bodyType rangoo
    df=datas.sort_values(by=["Count"],ascending=False)
    fig8 = px.pie(df, values="Count", names=df.columns.values.tolist()[0], title='Availability',hole=0.35) 
    fig8.update_traces(textfont_size=18,marker=dict(colors=['#02B3D5', '#008838']))#,line=dict(color=['white']*2, width=[0.5,0.5])

    return [fig, fig2,fig3,fig4,fig5,fig6,fig7,fig8]

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8059)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
